In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = x.view(-1, 10)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the genetic algorithm
def ga(model, train_loader, test_loader, num_generations=10, population_size=10):
    # Define the fitness function
    def fitness_fn(model, train_loader):
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001)
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.squeeze() # Squeeze the extra dimension from the outputs tensor
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        return running_loss / len(train_loader)

    # Initialize the population
    population = []
    for i in range(population_size):
        # Create a new model with random weights
        new_model = Net()
        for param in new_model.parameters():
            param.data = torch.randn_like(param.data)
        # Calculate the fitness of the new model
        fitness = fitness_fn(new_model, train_loader)
        # Add the new model and its fitness to the population
        population.append((new_model, fitness))

    # Run the genetic algorithm for the specified number of generations
    for generation in range(num_generations):
        # Sort the population by fitness (lowest to highest)
        population.sort(key=lambda x: x[1])

        # Select the top half of the population as parents for the next generation
        num_parents = population_size // 2
        parents = [parent for parent, fitness in population[:num_parents]]
        #parents = population[:population_size//2]

        # Create a new population by crossing over pairs of parents and mutating their offspring
        new_population = []
        for i in range(population_size):
            # Select two parents at random
            parent1, parent2 = np.random.choice(parents, size=2, replace=False)

            # Create a new model by crossing over the weights of the parents
            new_model = Net()
            for param1, param2, new_param in zip(parent1.parameters(), parent2.parameters(), new_model.parameters()):
                # Cross over the weights by taking the average of the parent weights
                new_param.data = (param1.data + param2.data) / 2

                # Mutate the weights by adding some random noise
                new_param.data += torch.randn_like(new_param.data) * 0.1

            # Calculate the fitness of the new model
            fitness = fitness_fn(new_model, train_loader)

            # Add the new model and its fitness to the new population
            new_population.append((new_model, fitness))

        # Replace the old population with the new population
        population = new_population

    # Return the best model from the final generation
    best_model = min(population, key=lambda x: x[1])[0]
    return best_model

import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

# Load the data from the CSV file into a DataFrame
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data = data.drop(["ID","Age","ZIP Code"],axis = 1)

# Split the data into features (X) and labels (y)
X = data.drop('Personal Loan', axis=1).values
y = data['Personal Loan'].values

# Convert the data into Tensors
X = torch.Tensor(X)
y = torch.Tensor(y)

# Create a TensorDataset from the data
dataset = TensorDataset(X, y)

# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders for the training and test sets
train_loader = DataLoader(train_data, batch_size=64)
test_loader = DataLoader(test_data)

# Create a neural network and optimize its weights using a genetic algorithm
model = Net()
model = ga(model, train_loader, test_loader)

# Test the accuracy of the optimized model on some sample data (e.g., MNIST test set)
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print('Accuracy: {:.2f}%'.format(accuracy * 100))

Accuracy: 91.90%


In [11]:
class CulAl:
    def __init__(self, num_inputs, num_hidden, num_outputs, num_knowledges, knowledge_threshold):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.num_knowledges = num_knowledges
        self.knowledge_threshold = knowledge_threshold
        self.hidden_weights = np.random.randn(self.num_inputs, self.num_hidden)
        self.output_weights = np.random.randn(self.num_hidden, self.num_outputs)
        self.knowledges = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        hidden_layer = self.sigmoid(np.dot(inputs, self.hidden_weights))
        output_layer = self.sigmoid(np.dot(hidden_layer, self.output_weights))
        return output_layer

    def calculate_fitness(self, X, y):
        predictions = self.predict(X)
        fitness = np.sum(np.abs(predictions - y))
        return fitness

    def update_knowledge(self, X, y):
        fitness = self.calculate_fitness(X, y)
        if len(self.knowledges) < self.num_knowledges or fitness < max([k["fitness"] for k in self.knowledges]):
            self.knowledges.append({"hidden_weights": self.hidden_weights.copy(), "output_weights": self.output_weights.copy(), "fitness": fitness})
        self.knowledges = sorted(self.knowledges, key=lambda k: k["fitness"])[:self.num_knowledges]

    def inherit_knowledge(self):
        if len(self.knowledges) > 0 and self.calculate_fitness(X, y) > self.knowledge_threshold:
            best_knowledge = self.knowledges[0]
            self.hidden_weights = best_knowledge["hidden_weights"].copy()
            self.output_weights = best_knowledge["output_weights"].copy()

    def train(self, X, y, population_size=10, generations=100, mutation_rate=0.1):
        population = [CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold) for _ in range(population_size)]

        for generation in range(generations):
            for individual in population:
                individual.update_knowledge(X, y)

            for individual in population:
                individual.inherit_knowledge()

            fitnesses = [individual.calculate_fitness(X, y) for individual in population]
            fittest_idx = np.argmin(fitnesses)
            fittest = population[fittest_idx]

            if fitnesses[fittest_idx] == 0:
                print("Solution found in generation", generation)
                break

            parents = np.random.choice(population, size=population_size-1, replace=True, p=np.array(fitnesses) / np.sum(fitnesses))
            children = [fittest.crossover(parent) for parent in parents]
            population = [fittest] + children
            for individual in population:
                individual.mutate(mutation_rate)

        self.hidden_weights = fittest.hidden_weights
        self.output_weights = fittest.output_weights

    def crossover(self, other):
        child = CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold)
        child.hidden_weights = self.hidden_weights.copy()
        child.output_weights = other.output_weights.copy()
        return child

    def mutate(self, mutation_rate):
        for i in range(self.num_inputs):
            for j in range(self.num_hidden):
                if np.random.random() < mutation_rate:
                    self.hidden_weights[i,j] += np.random.randn() * 0.1

        for i in range(self.num_hidden):
            for j in range(self.num_outputs):
                if np.random.random() < mutation_rate:
                    self.output_weights[i,j] += np.random.randn() * 0.1
                    
accuracy = correct / total

print('Accuracy: {:.2f}%'.format(accuracy * 100))

Accuracy: 66.59%
